In [ ]:
import sys
sys.path.insert(0, "../../../shared")
from utils import (load_program, print_program)

from chia.types.blockchain_format.program import Program

counter_puzzle = load_program("counter.clsp", ["../../../shared"])
terminate_singleton_puzzle = load_program("terminate-singleton.clsp", ["../../../shared"])
print(counter_puzzle.get_tree_hash())
print(terminate_singleton_puzzle.get_tree_hash())